In [1]:
import os 

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\end to end 2\\Chicken-Disease-end-to-end-'

In [4]:
import tensorflow as tf 

In [5]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaulationConfig:
      path_of_model: Path
      training_data : Path
      all_params : dict
      params_image_size:list
      params_batch_size: int 
      


In [7]:
from cnnClassfier.constants import * 
from cnnClassfier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager():

        def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

        def get_validation_config(self) -> EvaulationConfig:
            eval_config= EvaulationConfig(
                    path_of_model="artifacts/training/model.h5",
                    training_data="artifacts/data_ingestion/Chicken-fecal-images",
                    all_params=self.params,
                    params_image_size=self.params.IMAGE_SIZE,
                    params_batch_size=self.params.BATCH_SIZE,)

            return eval_config

In [9]:
from urllib.parse import urlparse

In [25]:
class Evaulation:
    def __init__(self,config:EvaulationConfig):
        self.config=config

    def _valid_generator(self):

        datagenerator_kwargs=dict(
            rescale= 1./255,
            validation_split=.30)
        
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear")
        
        
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs)
        

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs

        )
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)

    def save_score(self):
        scores={"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("score.josn"),data=scores) 


        

In [26]:
try:
    config= ConfigurationManager()
    val_config=config.get_validation_config()
    evaulation=Evaulation(val_config)
    evaulation.evaluation()
    evaulation.save_score()
except Exception as e :
    raise e

[2024-04-08 05:41:40,402: INFO: common: yaml file config\config.yaml load sucessfuly]
[2024-04-08 05:41:40,404: INFO: common: yaml file params.yaml load sucessfuly]
[2024-04-08 05:41:40,406: INFO: common: created dierctor ast artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 12s 1s/step - loss: 2.5499 - accuracy: 0.6638
[2024-04-08 05:41:52,600: INFO: common: json file saved at score.josn]
